In [2]:
!pip install transformers
!pip install dateparser
!pip install hanlp
!pip install regex

In [43]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import dateparser
import re
import hanlp
from datetime import datetime

In [44]:
# ===== 一、初始化兩套模型 =====
# 1. ckiplab NER
model_name = "ckiplab/bert-base-chinese-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# 2. HanLP
hanlp_pipeline = hanlp.load('FINE_ELECTRA_SMALL_ZH')


The following layers were not sharded: bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.weight, classifier.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.embeddings.LayerNorm.weight, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.self.query.bias, classifier.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.la

In [45]:
# ===== 二、分析函式 =====

def normalize_time(time_str, base_year=None):
    # 正規化「下午3點」等為24小時制
    if not time_str:
        return None
    year = base_year or datetime.now().year
    # 補年份
    time_str = re.sub(r"^(\d{1,2})月", f"{year}年\\1月", time_str)
    # AM/PM處理
    match = re.search(r"(下午|晚上|傍晚|中午|凌晨|早上|上午)?(\d{1,2})點半?", time_str)
    if match:
        h = int(match.group(2))
        if match.group(1) in ["下午", "晚上", "傍晚"]:
            if h < 12:
                h += 12
        elif match.group(1) in ["中午"]:
            h = 12
        elif match.group(1) in ["凌晨"]:
            if h == 12:
                h = 0
        # 替換原本的時間
        time_str = re.sub(r"(下午|晚上|傍晚|中午|凌晨|早上|上午)?(\d{1,2})點半?", f"{h:02d}:00", time_str)
    dt = dateparser.parse(time_str, languages=['zh'])
    return dt

def extract_location(sentence, hanlp_out):
    # 先用 HanLP
    if 'location' in hanlp_out and hanlp_out['location']:
        return hanlp_out['location'][0][0]
    # 若 HanLP沒抓到，用地點關鍵字正則補
    # 關鍵字可依需求自行擴充
    location_keywords = r"(車站|大樓|館|中心|醫院|公園|學校|飯店|超市|百貨|圖書館|機場|體育館|捷運站|火車站|商場|台北車站|工學大樓|大樓|早餐店)"
    # 抓「2~8字」+關鍵字，如「台北車站」、「信義大樓」
    loc_match = re.search(r"([\u4e00-\u9fa5]{2,8}" + location_keywords + ")", sentence)
    if loc_match:
        return loc_match.group(0)
    return ""

def clean_event_desc(sentence, time_str, location):
    desc = sentence
    if time_str:
        desc = desc.replace(time_str, "")
    if location:
        desc = desc.replace(location, "")
    # 去掉「我在」「要到」「參加」等助詞/動詞前綴
    desc = re.sub(r"^[我你他她在要去到於參加前往]+", "", desc)
    desc = desc.strip("，。 ")
    return desc


In [46]:
# ===== 三、主流程 (雙模型融合) =====
sentence = "我在6月10日下午3點要到台北車站參加論文口試"

# 1. transformers NER 預測
ner_results = ner(sentence)

# 2. HanLP 預測
hanlp_time, hanlp_location = extract_by_hanlp(sentence)

# 3. 正則抓時間
regex_time = extract_time_regex(sentence)

# 4. NER抓時間、地點
ner_time = extract_time_ner(sentence, ner_results)
ner_location = extract_location_ner(ner_results)


In [47]:
# ===== 四、採用「最穩定」來源：優先順序 HanLP > 正則 > NER =====
# 時間
time_str = hanlp_time or regex_time or ner_time
# 地點
location = hanlp_location or ner_location

# 事件描述（從原文移除時間、地點）
event_desc = extract_event_desc(sentence, time_str, location)

# 解析時間
parsed_time = dateparser.parse(time_str, languages=['zh'])

In [37]:
# ===== 五、最終輸出 =====
print("✅ 擷取結果：")
print("🕒 時間：", parsed_time, "｜原文：", time_str)
print("📍 地點：", location)
print("📝 事件描述：", event_desc)

✅ 擷取結果：
🕒 時間： None ｜原文： 禮拜二
📍 地點： 
📝 事件描述： 我們下去吃早餐


In [48]:


# 載入 HanLP
hanlp_pipeline = hanlp.load('FINE_ELECTRA_SMALL_ZH')

def extract_time_regex(sentence):
    """用正則式補捉中文時間"""
    patterns = [
        r"\d{1,2}月\d{1,2}日[上下]午\d{1,2}點半?",
        r"\d{1,2}月\d{1,2}日\d{1,2}點半?",
        r"\d{1,2}月\d{1,2}日\d{1,2}點",
        r"\d{1,2}月\d{1,2}日",
        r"[上下]午\d{1,2}點半?",
        r"\d{1,2}點半?",
        r"\d{1,2}點",
    ]
    regex = "|".join(patterns)
    match = re.search(regex, sentence)
    if match:
        return match.group()
    return ""

def normalize_time(time_str, base_year=None):
    """將時間文字轉成 datetime"""
    if not time_str:
        return None
    year = base_year or datetime.now().year
    time_str = re.sub(r"^(\d{1,2})月", f"{year}年\\1月", time_str)
    # 轉換下午、晚上等詞彙
    match = re.search(r"(下午|晚上|傍晚|中午|凌晨|早上|上午)?(\d{1,2})點(半)?", time_str)
    if match:
        h = int(match.group(2))
        if match.group(1) in ["下午", "晚上", "傍晚"] and h < 12:
            h += 12
        elif match.group(1) == "中午":
            h = 12
        elif match.group(1) == "凌晨" and h == 12:
            h = 0
        m = "30" if match.group(3) else "00"
        time_str = re.sub(r"(下午|晚上|傍晚|中午|凌晨|早上|上午)?(\d{1,2})點(半)?", f"{h:02d}:{m}", time_str)
    dt = dateparser.parse(time_str, languages=['zh'])
    return dt

def extract_location(sentence, hanlp_out):
    """地點抽取：HanLP + 自訂關鍵字後綴"""
    if 'location' in hanlp_out and hanlp_out['location']:
        return hanlp_out['location'][0][0]
    location_keywords = r"(車站|大樓|館|中心|醫院|公園|學校|飯店|超市|百貨|圖書館|機場|體育館|捷運站|火車站|商場|台北車站)"
    loc_match = re.search(r"[\u4e00-\u9fa5]{2,10}" + location_keywords, sentence)
    if loc_match:
        return loc_match.group(0)
    return ""

def clean_event_desc(sentence, time_str, location):
    """移除時間與地點，留下事件描述"""
    desc = sentence
    if time_str:
        desc = desc.replace(time_str, "")
    if location:
        desc = desc.replace(location, "")
    desc = re.sub(r"^[我你他她在要去到於參加前往]+", "", desc)
    return desc.strip("，。 ")

def extract_event(sentence):
    """主抽取流程：時間、地點、描述"""
    hanlp_out = hanlp_pipeline(sentence)

    time_str = hanlp_out['time'][0][0] if 'time' in hanlp_out and hanlp_out['time'] else extract_time_regex(sentence)
    parsed_time = normalize_time(time_str)

    location = extract_location(sentence, hanlp_out)
    event_desc = clean_event_desc(sentence, time_str, location)

    return {
        "時間原文": time_str,
        "時間解析": parsed_time,
        "地點": location,
        "事件描述": event_desc
    }

# ==== 測試 ====
sentence = "我在6月10日下午3點要到台北車站參加論文口試"
event_info = extract_event(sentence)

print("✅ 擷取結果：")
print("🕒 時間：", event_info["時間解析"], "｜原文：", event_info["時間原文"])
print("📍 地點：", event_info["地點"])
print("📝 事件描述：", event_info["事件描述"])


✅ 擷取結果：
🕒 時間： 2025-06-10 15:00:00 ｜原文： 6月10日下午3點
📍 地點： 點要到台北車站
📝 事件描述： 台北車站參加論文口試
